In [1]:
import numpy as np
import time

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

import xgboost as xg

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [2]:
def lr_train(X, y):
  lr = LogisticRegression()
  lr.fit(X, y)
  return lr

def svc_train(X, y):
  svc = SVC()
  svc.fit(X, y)
  return svc

def xgb_train(X, y):
  xgb = xg.XGBClassifier()
  xgb.fit(X, y)
  return xgb

# SGD Training
# w0 = np.zeros((n+1, 1))  # The estimated value of w.
# t0 = time.process_time()
# for i in range(n+1):
#     w0[i] = np.random.rand()*0.1 - 0.05

# eta = 0.01  # learning rate
# for t in range(100):
#     for i in range(training_size):
#         c = X[i:i+1,:]
#         r = y[i]
#         phi = np.ones(n+1)
#         phi[n] = 1
#         for j in range(n-1, -1, -1):
#             phi[j] = (2*c[0,j]-1)*phi[j+1]

#         h = (np.dot(phi, w0) > 0)
#         e = r - h
#         delta_w = eta * e * phi.reshape(n+1,1)
#         w0 = w0 + delta_w

In [3]:
def compute_phi(c):
  n = c.shape[-1]
  phi = np.ones(n+1)
  phi[n] = 1
  for i in range(n-1, -1, -1):
    phi[i] = (2*c[0,i]-1)*phi[i+1]

  return phi

def puf_init(n=64):
  np.random.seed(int(time.time()))
  data = np.loadtxt('weight_diff.txt')
  w = np.zeros((n+1, 1))
  for i in range(1, n+2):
    randi_offset = np.random.randint(1, 45481)
    w[i-1] = data[randi_offset-1]

  return w

def puf_query(c, w, debug=False):
  phi = compute_phi(c)
  r = (np.dot(phi, w) > 0)
  if debug:
    print("Phi: ", phi, "W: ", w, sep="\n")

  return r

def make_samples(n, w):
  c = np.random.randint(0, 2, size=(n, 1, w.shape[0]-1))
  r = np.array([puf_query(i, w) for i in c])

  return c, r

def train(train_fn, X, y):
  t0 = time.process_time()
  # ADD YOUR TRAINING CODE HERE

  model = train_fn(X, y)

  t1 = time.process_time()
  training_time = t1 - t0                             # time taken to get w0
  print("Training time:", training_time)
  print("Training size:", training_size)

  return model, training_time

def eval(model, training_time, n=64):
  n_test = 10000
  correct = 0
  for i in range(1, n_test+1):
    c_test = np.random.randint(0, 2, size=(1, n))     # a random challenge vector
    r = puf_query(c_test, w)
    c_test = compute_phi(c_test).reshape(1, -1)
    r0 = model.predict(c_test)
    correct += (r==r0)

  success_rate = correct/n_test
  print("Success rate:", success_rate)

  # If the success rate is less than 99%, a penalty time will be added
  # One second is add for each 0.01% below 99%.
  effective_training_time = training_time
  if success_rate < 0.99:
      effective_training_time = training_time + 10000*(0.99-success_rate)
  print("Effective training time:", effective_training_time)

# Problem Setup
target = 0.99                                        # The desired prediction rate
n = 64                                               # number of stages in the PUF

# Initialize the PUF
w = puf_init(n)

# You can use the puf_query function to generate your training dataset
# ADD YOUR DATASET GENERATION CODE HERE
training_size = 10000

X, y = make_samples(training_size, w)
X = np.array([compute_phi(i) for i in X])

for i in [
    {"model": svc_train, "name": "SVC"}, 
    {"model": lr_train, "name": "Log Reg"}, 
    {"model": xgb_train, "name": "XG Boosting"}]:
  print("\n", "-"*20, i["name"], "-"*20, sep=" ", end="\n")
  model, train_time = train(i["model"], X, y.ravel())
  eval(model, train_time)


 -------------------- SVC --------------------
Training time: 4.091682948000001
Training size: 10000
Success rate: [0.9664]
Effective training time: [240.09168295]

 -------------------- Log Reg --------------------
Training time: 0.09776696400000162
Training size: 10000
Success rate: [0.9947]
Effective training time: 0.09776696400000162

 -------------------- XG Boosting --------------------
Training time: 4.3665228379999945
Training size: 10000
Success rate: [0.907]
Effective training time: [834.36652284]


In [4]:
class Data(Dataset):
    def __init__(self, X, y):
        self.X = torch.from_numpy(X.astype(np.float32))
        self.y = torch.from_numpy(y.astype(np.float32))
        self.len = self.X.shape[0]
       
    def __getitem__(self, index):
        return self.X[index], self.y[index]
   
    def __len__(self):
        return self.len

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(65, 65)
        self.a1 = nn.ReLU()
        self.fc2 = nn.Linear(65, 65)
        self.a2 = nn.ReLU()
        self.fc3 = nn.Linear(65, 1)

    def forward(self, x):
        out = self.fc1(x)
        out = self.a1(out)
        out = self.fc2(out)
        out = self.a2(out)
        out = self.fc3(out)
        return out


model = Net()
print(model)
     


batch_size = 64

# Instantiate training and test data
train_data = Data(X, y)
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

learning_rate = 0.0001

loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Net(
  (fc1): Linear(in_features=65, out_features=65, bias=True)
  (a1): ReLU()
  (fc2): Linear(in_features=65, out_features=65, bias=True)
  (a2): ReLU()
  (fc3): Linear(in_features=65, out_features=1, bias=True)
)


In [5]:
num_epochs = 100
loss_values = []

t0 = time.process_time()

for epoch in range(num_epochs):
    for X, y in train_dataloader:
        optimizer.zero_grad()
        pred = model(X)
        loss = loss_fn(pred, y)
        loss_values.append(loss.item())
        loss.backward()
        optimizer.step()
        # print("Train Loss {:.4f}".format(loss.item()))

t1 = time.process_time()

print("Training Complete\nTime: {}\nAvg. Loss: {}".format(t1-t0, sum(loss_values)/len(loss_values)))

n_test = 10000
correct = 0

model.eval()

with torch.no_grad():
  for i in range(1, n_test+1):
      c_test = np.random.randint(0, 2, size=(1, n))
      r = puf_query(c_test, w)
      c_test = torch.from_numpy(compute_phi(c_test).astype(np.float32))
      r0 = int(model(c_test) > 0)
      correct += (r==r0)

success_rate = correct/n_test
print("Success rate:", success_rate)

Training Complete
Time: 20.959859390999995
Avg. Loss: 0.6914078287561988
Success rate: [0.5284]
